In [23]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

In [24]:
car = pd.read_csv('true_car_listings.csv')


In [25]:
car

,Price,Year,Mileage,City,State,Vin,Make,Model
0,8995,2014,35725,El Paso,TX,19VDE2E53EE000083,Acura,ILX6-Speed
1,10888,2013,19606,Long Island City,NY,19VDE1F52DE012636,Acura,ILX5-Speed
2,8995,2013,48851,El Paso,TX,19VDE2E52DE000025,Acura,ILX6-Speed
3,10999,2014,39922,Windsor,CO,19VDE1F71EE003817,Acura,ILX5-Speed
4,14799,2016,22142,Lindon,UT,19UDE2F32GA001284,Acura,ILXAutomatic
...,...,...,...,...,...,...,...,...
852117,63215,2017,9,Culver City,CA,YV1A22MK9H1013237,Volvo,S90T6
852118,72260,2017,3201,Englewood,NJ,YV4A22PL3H1186162,Volvo,XC90T6
852119,55999,2016,28941,Fort Collins,CO,YV4A22PL4G1000868,Volvo,XC90AWD
852120,60240,2017,3005,San Leandro,CA,YV4A22NLXH1006162,Volvo,V90


In [26]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852122 entries, 0 to 852121
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Price    852122 non-null  int64 
 1   Year     852122 non-null  int64 
 2   Mileage  852122 non-null  int64 
 3   City     852122 non-null  object
 4   State    852122 non-null  object
 5   Vin      852122 non-null  object
 6   Make     852122 non-null  object
 7   Model    852122 non-null  object
dtypes: int64(3), object(5)
memory usage: 52.0+ MB


In [27]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [28]:
x = car.iloc[:, [1,2,3,4,6,7]]

In [29]:
x

,Year,Mileage,City,State,Make,Model
0,2014,35725,El Paso,TX,Acura,ILX6-Speed
1,2013,19606,Long Island City,NY,Acura,ILX5-Speed
2,2013,48851,El Paso,TX,Acura,ILX6-Speed
3,2014,39922,Windsor,CO,Acura,ILX5-Speed
4,2016,22142,Lindon,UT,Acura,ILXAutomatic
...,...,...,...,...,...,...
852117,2017,9,Culver City,CA,Volvo,S90T6
852118,2017,3201,Englewood,NJ,Volvo,XC90T6
852119,2016,28941,Fort Collins,CO,Volvo,XC90AWD
852120,2017,3005,San Leandro,CA,Volvo,V90


In [30]:
y = car.iloc[:, 0]

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=3)

In [32]:
ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(x[['City','State','Make','Model']])

OneHotEncoder(handle_unknown='ignore')

In [33]:
transform = make_column_transformer((OneHotEncoder(handle_unknown='ignore',categories=ohe.categories_),['City','State','Make','Model']),
                                    (StandardScaler(),['Year','Mileage'])
                                    ,remainder='drop')

In [34]:
lr = LinearRegression()

In [35]:
pipe = make_pipeline(transform, lr)

In [36]:
pipe.fit(x_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['AKRON', 'ALEXANDRIA', 'AURORA', ..., 'phoenix', 'salt lake',
       'victorville'], dtype=object),
                                                                            array([' AK', ' AL', ' AR', ' AZ', ' Az', ' CA', ' CO', ' CT', ' Ca',
       ' DC', ' DE', ' FL', ' Fl', ' GA', ' Ga', ' HI', ' IA', ' ID',
       ' IL', ' IN', ' KS', ' KY', ' LA', ' MA', ' MD', ' ME', ' MI',
       ' MN',...
       'Pontiac', 'Porsche', 'Ram', 'Rolls-Royce', 'Saab', 'Saturn',
       'Scion', 'Subaru', 'Suzuki', 'Tesla', 'Toyota', 'Volkswagen',
       'Volvo', 'smart'], dtype=object),
                                                                            array(['1', '124', '1500', ..., 'xD5dr', 'xDAutomatic', 'xDManual'],
      dtype=object)],
                                                                handle_unknown='ignore'),
                                                  ['City', 'State', 'Make',
                                                   'Model']),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  ['Year', 'Mileage'])])),
                ('linearregression', LinearRegression())])

In [37]:
y_pred = pipe.predict(x_test)

In [38]:
y_pred

array([19725.06459298, 34860.78316147, 35412.53795262, ...,
       21904.7523686 , 36714.65245095, 24825.29785698])

In [39]:
r2_score(y_test, y_pred)

0.871523409957906

In [40]:
from sklearn.metrics import r2_score, mean_absolute_error

y_pred = pipe.predict(x_test)

print("R2 Score:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))

R2 Score: 0.871523409957906
MAE: 2851.829076449592


In [41]:
len(lr.coef_)

5408

In [42]:
y_pred = pipe.predict(x_test)
print("R2 Score:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))

R2 Score: 0.871523409957906
MAE: 2851.829076449592


In [43]:
# from sklearn.model_selection import cross_val_score
# cross_val_score(pipe, x, y, scoring='r2', cv=5).mean()

In [44]:
# import matplotlib.pyplot as plt
# import seaborn as sns

In [45]:
# plt.figure(figsize=(12, 8))
# sns.scatterplot(x=y_test, y=y_pred)
# plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
# color = 'red', linestyle='--')
# plt.xlabel('Actual Price')
# plt.ylabel('Predicted Price')
# plt.show()

In [46]:
# sns.histplot(data=car, x = 'Price', bins = 30,kde = True)
# plt.show()

In [47]:
# sns.scatterplot(x=y_test, y=y_pred)

In [48]:
# for i in range(15):
#     x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
#     lr = LinearRegression()
#     pipe = make_pipeline(transform, lr)
#     pipe.fit(x_train, y_train)
#     y_pred = pipe.predict(x_test)
#     print("R2 Score:",i,'-', r2_score(y_test, y_pred))

In [49]:
from sklearn.metrics import r2_score

# Regular R² score
r2 = r2_score(y_test, y_pred)

# n = number of samples, p = number of features
n = x_test.shape[0]
p = x_test.shape[1]

# Adjusted R² formula
adjusted_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

print("R² Score:", r2)
print("Adjusted R² Score:", adjusted_r2)


R² Score: 0.871523409957906
Adjusted R² Score: 0.8715188866121312


In [50]:
x_test.shape[0]

170425

In [51]:
x_test.shape[1]

6

In [52]:
x_test

,Year,Mileage,City,State,Make,Model
461287,2015,43947,Seekonk,MA,Hyundai,Santa
7586,2018,10,Langhorne,PA,Acura,RDXAWD
816201,2015,20445,Manchester,NH,Toyota,Tundra
492460,2016,9889,Latham,NY,Jeep,RenegadeJustice
575571,2014,47479,Marietta,GA,Lincoln,MKZFWD
...,...,...,...,...,...,...
815517,2016,17230,Edmonds,WA,Toyota,HighlanderAWD
153725,2014,46373,Durant,OK,Chevrolet,Silverado
121466,2010,95591,Muscle Shoals,AL,Chevrolet,Silverado
316969,2016,19545,Price,UT,Ford,F-1504WD


In [55]:
pipe.predict(pd.DataFrame([[2010,50000,'Seekonk','GA','Ford','EscapeSE']], columns=['Year','Mileage','City','State','Make','Model']))

array([10049.47875835])

In [54]:
# import pickle
# pickle.dump(pipe, open('LinearReggrationModel.pkl', 'wb'))